In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean

# creamos una sesión de spark
spark = SparkSession.builder.appName("ParquetReader").getOrCreate()

# columnas que nos queremos quedar
columns = ['passenger_count', 'tip_amount']

# lista vacia para ir juntando los dfs
df_list = []

# levantamos cada archivo
for i in range(1, 13):
    file_name = f'yellow_tripdata_2015-{i:02}.parquet'
    df = spark.read.format("parquet").load(file_name).select(columns)
    df_list.append(df)

# juntamos todo en un dataframe grande
big_df = df_list[0]
for i in range(1, len(df_list)):
    big_df = big_df.union(df_list[i])

# calculate the mean tip amount per passenger count
mean_tip_amount = big_df.groupBy("passenger_count").agg(mean("tip_amount"))

# show the results
mean_tip_amount.show()


+---------------+------------------+
|passenger_count|   avg(tip_amount)|
+---------------+------------------+
|              0|1.5967455694024544|
|              7| 4.067709251101321|
|              6|1.6809094828879472|
|              9| 5.198969696969696|
|              5| 1.728789283610509|
|              1|1.7529720022452289|
|              3|1.5800112230844956|
|              8| 5.146011235955056|
|              2|1.7061289523911716|
|              4|1.4594104637872471|
+---------------+------------------+

